## 套件

In [1]:
import pandas as pd
import statsmodels.api as sm

In [2]:
# price_data = pd.read_csv('../data/stock_100_price.txt',  encoding='cp950')
# earn_data = pd.read_csv('../data/stock_100_earn.txt',  encoding='cp950')
# institution_data = pd.read_csv('../data/stock_100_institution.txt',  encoding='cp950')
# bs_data = pd.read_csv('../data/stock_100_bs.txt',  encoding='cp950')

price_data = pd.read_csv('../data/price.txt',  encoding='cp950')
earn_data = pd.read_csv('../data/ifrs.txt',  encoding='cp950')
institution_data = pd.read_csv('../data/institution.txt',  encoding='cp950')
bs_data = pd.read_csv('../data/margin.txt',  encoding='cp950')

/Users/gy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/gy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# 查看前五筆資料
price_data.head()

,證券代碼,簡稱,TEJ產業_名稱,TSE新產業_名稱,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),報酬率％,週轉率％,市值(百萬元),股價淨值比-TSE
0,1101,台泥,水泥製造,水泥工業,20170103,35.45,35.45,34.80,35.15,2890,0.0000,0.0783,129779,1.23
1,1102,亞泥,水泥製造,水泥工業,20170103,26.45,26.50,26.30,26.50,1271,0.5693,0.0378,89078,0.73
2,1103,嘉泥,水泥製造,水泥工業,20170103,8.71,8.71,8.61,8.71,278,-0.2291,0.0359,6748,0.4
3,1104,環泥,水泥製造,水泥工業,20170103,24.50,24.65,24.45,24.60,150,-0.2028,0.0236,15610,0.97
4,1108,幸福,水泥製造,水泥工業,20170103,9.13,9.15,9.13,9.15,20,0.3289,0.0049,3703,0.78


In [4]:
# 查看後五筆資料
price_data.tail()

,證券代碼,簡稱,TEJ產業_名稱,TSE新產業_名稱,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),報酬率％,週轉率％,市值(百萬元),股價淨值比-TSE
2129617,9949,琉園,玻璃陶瓷,文化創意業,20220325,12.85,12.9,12.85,12.90,3,0.0000,0.0068,568,1.31
2129618,9950,萬國通,鞋及旅行箱,塑膠工業,20220325,11.90,11.9,11.85,11.90,44,0.0000,0.0298,1758,3.93
2129619,9951,皇田,汽車組件,電機機械,20220325,74.00,74.0,73.50,73.80,32,-0.2703,0.0427,5528,1.78
2129620,9960,邁達康,百貨批發,貿易百貨,20220325,25.55,25.8,25.55,25.75,22,0.9804,0.0655,865,1.72
2129621,9962,有益,金屬基本,鋼鐵工業,20220325,22.05,22.8,21.10,21.95,21617,1.1521,23.9603,1980,1.82


In [5]:
rename_column_maplist = {
    '證券代碼' : "code", 
    '公司': 'code',
    '證券名稱': 'code',
    '公司代碼': 'code',
    
    '簡稱' : 'name', 
    'TSE新產業_名稱' : 'new_industry', 
    'TEJ產業_名稱' : 'industry', 
    '年月日'  : "date", 
    '年/月': 'date_year_month',
    '收盤價(元)' : 'close', 
    '成交量(千股)' : 'volume', 
    '成交值(千元)' : 'amount',
    '報酬率％' : 'return', 
    '週轉率％' : "turn_over", 
    '市值(百萬元)' : "size", 
    '股價淨值比-TSE': "PB_TSE", 
    '股價淨值比-TEJ' : 'PB_TEJ',
    
    ### earn
    'ROA－綜合損益': 'ROA',
    '營收成長率': 'YOY',
    '總資產成長率': "TAR",
    
    ### in          
    '外資週轉率%' : 'foreign_turnover_rate',
    '投信週轉率%' : 'trust_turnover_rate',
    '自營商週轉率%' : 'broker_turnover_rate',
    
    ## bs
    '券資比': 'short_margin_ratio',
    '融資餘額(張)' : 'margin_balance',
    '融券餘額(張)' : 'short_balance',
}

In [6]:
print('欄位改名前：', price_data.columns)
price_data.rename(columns=rename_column_maplist, inplace=True)
print('欄位改名後：', price_data.columns)

欄位改名前： Index(['證券代碼', '簡稱', 'TEJ產業_名稱', 'TSE新產業_名稱', '年月日', '開盤價(元)', '最高價(元)',
       '最低價(元)', '收盤價(元)', '成交量(千股)', '報酬率％', '週轉率％', '市值(百萬元)', '股價淨值比-TSE'],
      dtype='object')
欄位改名後： Index(['code', 'name', 'industry', 'new_industry', 'date', '開盤價(元)', '最高價(元)',
       '最低價(元)', 'close', 'volume', 'return', 'turn_over', 'size', 'PB_TSE'],
      dtype='object')


In [7]:
# https://s61160230.pixnet.net/blog/post/218812467
print('欄位改名前：', earn_data.columns)
earn_data.rename(columns=rename_column_maplist, inplace=True)
print('欄位改名後：', earn_data.columns)

欄位改名前： Index(['公司', '簡稱', '年/月', '營收成長率', '總資產成長率'], dtype='object')
欄位改名後： Index(['code', 'name', 'date_year_month', 'YOY', 'TAR'], dtype='object')


In [8]:
print('欄位改名前：', institution_data.columns)
institution_data.rename(columns=rename_column_maplist, inplace=True)
print('欄位改名後：', institution_data.columns)

欄位改名前： Index(['證券名稱', '簡稱', '年月日', '外資週轉率%', '投信週轉率%', '自營商週轉率%'], dtype='object')
欄位改名後： Index(['code', 'name', 'date', 'foreign_turnover_rate', 'trust_turnover_rate',
       'broker_turnover_rate'],
      dtype='object')


In [9]:
print('欄位改名前：', bs_data.columns)
bs_data.rename(columns=rename_column_maplist, inplace=True)
print('欄位改名後：', bs_data.columns)

欄位改名前： Index(['公司代碼', '簡稱', '年月日', '券資比'], dtype='object')
欄位改名後： Index(['code', 'name', 'date', 'short_margin_ratio'], dtype='object')


# 大表

In [10]:
price_data['code'] = price_data['code'].apply(lambda x: str(x).strip())
institution_data['code'] = institution_data['code'].apply(lambda x: str(x).strip())
bs_data['code'] = bs_data['code'].apply(lambda x: str(x).strip())
earn_data['code'] = earn_data['code'].apply(lambda x: str(x).strip())


price_data['name'] = price_data['name'].apply(lambda x: str(x).strip())
institution_data['name'] = institution_data['name'].apply(lambda x: str(x).strip())
bs_data['name'] = bs_data['name'].apply(lambda x: str(x).strip())
earn_data['name'] = earn_data['name'].apply(lambda x: str(x).strip())

price_data['new_industry'] = price_data['new_industry'].apply(lambda x: str(x).strip())
price_data['industry'] = price_data['industry'].apply(lambda x: str(x).strip())

In [11]:
merge_data = price_data.merge(institution_data, on=['code', 'name', 'date'])\
.merge(bs_data, on=['code', 'name', 'date'])
merge_data['date_year_month']= merge_data['date'].apply(lambda the_date: int(str(the_date)[:6]))

merge_data['code'] = merge_data['code'].apply(lambda x: str(x).strip())
earn_data['code'] = earn_data['code'].apply(lambda x: str(x).strip())

all_data = merge_data.merge(
    earn_data, on=['code', 'name', 'date_year_month'], how='outer').sort_values(by='date_year_month')

# 迴圈測試

In [12]:
from tqdm import tqdm

code_list = all_data['code'].drop_duplicates().values

In [13]:
base_x_variable_columns = [
        'return', 
        'size', 
        'turn_over', 
        'PB_TSE',
]

ifrs_x_variable_columns = [
        'return', 
        'size', 
        'turn_over', 
        'PB_TSE',
    
        'YOY', 
        'TAR',
]


all_x_variable_columns = [
        'return', 
        'size', 
        'turn_over', 
        'PB_TSE',
    
        'YOY', 
        'TAR',    
    
        'margin_short_ratio', 
        'intitution_turnover_rate',
]



In [14]:
error_list = []
res_list = []
empty_list = []

for code in tqdm(code_list):
    stock_data = all_data.query('code == @code')

    stock_data = stock_data.ffill()
    stock_data['return_t1'] = stock_data['return'].shift(-1)

    stock_data = stock_data.dropna()

    if stock_data.empty:
        empty_list.append(code)
        continue

    try:

        stock_data['PB_TSE'] = stock_data['PB_TSE'].astype(float)
        stock_data['YOY'] = stock_data['YOY'].astype(float)
        stock_data['TAR'] = stock_data['TAR'].astype(float)

        # 資券比
        stock_data['short_margin_ratio'] = stock_data['short_margin_ratio'].astype(
            float)

        def convert(x):
            if x == 0:
                return 0
            else:
                return 1/(x)*10000
        stock_data['margin_short_ratio'] = stock_data['short_margin_ratio'].apply(
            convert)

        # 合計週轉率
        stock_data['intitution_turnover_rate'] = stock_data[[
            'trust_turnover_rate', 'broker_turnover_rate', 'short_margin_ratio']].sum(1)

    except:
        #print("有問題：", code)
        error_list.append(code)
        continue

    stock_info = stock_data[["code", "name",
                             "new_industry", "industry"]].dropna().iloc[-1, :4]
    mean_size = stock_data['size'].mean()

    y = stock_data['return_t1'].values.reshape((-1, 1))
    x = stock_data[base_x_variable_columns]
    model = sm.OLS(y,  sm.add_constant(x,  prepend=False))
    res = model.fit()       # Fit model
    aviable_t = (res.pvalues < 0.1).sum()
    res_list.append({
        'code': code,
        'res': res,
        'name': stock_info['name'],
        'type': 'base',
        'aviable_t': aviable_t,
        'mean_size': mean_size,
        'new_industry': stock_info['new_industry'],
        'industry': stock_info['industry'],
    })

    y = stock_data['return_t1'].values.reshape((-1, 1))
    x = stock_data[ifrs_x_variable_columns]
    model = sm.OLS(y,  sm.add_constant(x,  prepend=False))
    res = model.fit()       # Fit model
    aviable_t = (res.pvalues < 0.1).sum()
    res_list.append({
        'code': code,
        'res': res,
        'name': stock_info['name'],
        'type': 'ifrs',
        'aviable_t': aviable_t,
        'mean_size': mean_size,
        'new_industry': stock_info['new_industry'],
        'industry': stock_info['industry'],
    })

    y = stock_data['return_t1'].values.reshape((-1, 1))
    x = stock_data[all_x_variable_columns]
    model = sm.OLS(y,  sm.add_constant(x,  prepend=False))
    res = model.fit()       # Fit model
    aviable_t = (res.pvalues < 0.1).sum()
    res_list.append({
        'code': code,
        'res': res,
        'name': stock_info['name'],
        'type': 'all',
        'aviable_t': aviable_t,
        'mean_size': mean_size,
        'new_industry': stock_info['new_industry'],
        'industry': stock_info['industry'],
    })

  0%|          | 0/1766 [00:00<?, ?it/s]/Users/gy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
/Users/gy/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:130: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
  2%|▏         | 43/1766 [00:04<02:18, 12.42it/s]/Users/gy/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
100%|██████████| 1766/1766 [02:37<00:00, 11.23it/s]


In [15]:
res_table = pd.DataFrame(res_list)

In [16]:
base_table = res_table.query('type == "base"').sort_values('aviable_t', ascending=False)
ifrs_table = res_table.query('type == "ifrs"').sort_values('aviable_t', ascending=False)
all_table = res_table.query('type == "all"').sort_values('aviable_t', ascending=False)

In [17]:
import numpy as np

# 考慮 基本市場資料

In [18]:
base_x_variable_columns

['return', 'size', 'turn_over', 'PB_TSE']

In [19]:
print('base vaar : ', len(base_x_variable_columns))
base_pivot_table = base_table.pivot_table(index=['new_industry'], aggfunc=np.mean)\
.sort_values(
    'aviable_t', ascending=False
)
base_pivot_table

base vaar :  4


,aviable_t,mean_size
new_industry,,
航運業,2.000000,28733.195616
農業科技,2.000000,2445.429020
電器電纜,2.000000,9601.824640
光電業,1.924051,17324.754878
鋼鐵工業,1.823529,20250.321153
資訊服務業,1.761905,3729.338571
建材營造,1.741935,8388.184740
電子通路業,1.678571,10384.190755
電子零組件,1.653595,13762.538337


In [20]:
target_new_industry = base_pivot_table.query('aviable_t >= 1.6').index.values

target_industry = base_table[base_table['new_industry'].isin(
    target_new_industry)
]
target_industry.pivot_table(
    index=['new_industry', 'industry'], 
    aggfunc=np.mean
)

aviable_t     mean_size
new_industry industry                         
光電業          主機板        1.000000  21343.393725
             光電/ IO     1.928571  12070.753810
             家電製品       2.000000    701.030733
             消費性電子      2.500000  62956.354980
             電子設備       1.571429   2869.000434
...                          ...           ...
電子零組件        資訊通路       2.600000   3805.432314
             通訊設備       2.000000   1388.392941
             金屬製品       1.000000  35498.682353
             電子設備       1.800000   3452.065510
             電子零組件      1.758242  13725.857049

[65 rows x 2 columns]

# 考慮 基本市場資料 ＋ 財務

In [21]:
ifrs_x_variable_columns

['return', 'size', 'turn_over', 'PB_TSE', 'YOY', 'TAR']

In [22]:
print('ifrs vaar : ', len(ifrs_x_variable_columns))
ifrs_pivot_table = ifrs_table.pivot_table(index=['new_industry'], aggfunc=np.mean)\
.sort_values(
    'aviable_t', ascending=False
)
ifrs_pivot_table

ifrs vaar :  6


,aviable_t,mean_size
new_industry,,
造紙工業,3.200000,20150.920627
航運業,2.409091,28733.195616
鋼鐵工業,2.235294,20250.321153
電器電纜,2.166667,9601.824640
農業科技,2.000000,2445.429020
光電業,1.962025,17324.754878
建材營造,1.919355,8388.184740
電子零組件,1.856209,13762.538337
其他電子業,1.830508,36034.103553


In [23]:
target_new_industry = ifrs_pivot_table.query('aviable_t >= 2').index.values

target_industry = ifrs_table[ifrs_table['new_industry'].isin(
    target_new_industry)
]
target_industry.pivot_table(
    index=['new_industry', 'industry'], 
    aggfunc=np.mean
)

aviable_t     mean_size
new_industry industry                         
航運業          機車船製造      0.500000  21610.160713
             海運         3.000000  42961.065465
             空運業        1.500000  69469.872941
             貨運倉儲業      2.444444   7035.627451
農業科技         藥          2.000000   2445.429020
造紙工業         紙漿         5.000000  13881.133333
             造紙         2.750000  21718.367451
鋼鐵工業         機車船製造      0.000000   3285.462745
             金屬基本       2.250000  21379.912819
             金屬製品       4.000000   1068.246275
電器電纜         家電製品       1.500000   5357.347149
             電線         2.500000  11724.063386

# 考慮 基本市場資料 ＋ 財務 + 投資人情緒

In [24]:
all_x_variable_columns

['return',
 'size',
 'turn_over',
 'PB_TSE',
 'YOY',
 'TAR',
 'margin_short_ratio',
 'intitution_turnover_rate']

In [25]:
print('all vaar : ', len(all_x_variable_columns))
all_pivot_table = all_table.pivot_table(index=['new_industry'], aggfunc=np.mean)\
.sort_values(
    'aviable_t', ascending=False
)
all_pivot_table

all vaar :  8


,aviable_t,mean_size
new_industry,,
造紙工業,3.400000,20150.920627
電子商務,3.333333,9334.056732
航運業,2.545455,28733.195616
玻璃陶瓷,2.400000,12382.547608
建材營造,2.338710,8388.184740
塑膠工業,2.333333,92878.873772
光電業,2.316456,17324.754878
其他電子業,2.271186,36034.103553
鋼鐵工業,2.264706,20250.321153


In [26]:
target_new_industry = all_pivot_table.query('aviable_t >= 2.25').index.values

target_industry = all_table[all_table['new_industry'].isin(
    target_new_industry)
]
target_industry.pivot_table(
    index=['new_industry', 'industry'], 
    aggfunc=np.mean
)

aviable_t      mean_size
new_industry industry                          
光電業          主機板        2.000000   21343.393725
             光電/ IO     2.357143   12070.753810
             家電製品       2.000000     701.030733
             消費性電子      2.400000   62956.354980
             電子設備       1.857143    2869.000434
             電子零組件      2.500000    5250.611176
其他電子業        PC系統       2.000000  700945.916468
             主機板        2.333333    3579.427758
             光電/ IO     2.875000    3050.472549
             其他電子       1.250000    7240.137451
             汽車組件       4.000000   11957.381176
             消費性電子      2.400000    2729.039598
             石化         2.333333    2082.731503
             資源回收       4.000000    1570.307451
             資訊通路       2.500000    7488.702796
             醫療耗材       5.000000    1736.880784
             電子設備       2.235294    9983.614660
             電子零組件      1.250000   51275.754412
塑膠工業         不織布        6.000000    3951.157647
             塑膠製品       2.500000    2807.059216
             石化         2.111111  107827.281841
建材營造         其他水泥       2.000000    1892.473725
             建設         2.240000    8724.637462
             營造         2.800000    6297.010312
             預拌混凝土      3.000000   18973.003922
玻璃陶瓷         玻璃陶瓷       2.400000   12382.547608
航運業          機車船製造      0.500000   21610.160713
             海運         3.000000   42961.065465
             空運業        2.000000   69469.872941
             貨運倉儲業      2.666667    7035.627451
造紙工業         紙漿         5.000000   13881.133333
             造紙         3.000000   21718.367451
鋼鐵工業         機車船製造      0.000000    3285.462745
             金屬基本       2.343750   21379.912819
             金屬製品       2.000000    1068.246275
電器電纜         家電製品       1.500000    5357.347149
             電線         2.625000   11724.063386
電子商務         軟體服務       5.000000    1588.912941
             通訊設備       2.500000   13206.628627
食品工業         其他食品       2.555556   10757.353638
             油脂飼料       2.200000   16249.486588
             罐頭加工       1.000000    4021.747451
             飲料乳品       2.000000  106515.680000

# 如果只看投資人情緒有影響的

In [27]:
all_table['is_margin_short_ratio_significant'] = all_table['res'].apply(lambda res: res.pvalues['margin_short_ratio'] <= 0.1 )
all_table['is_intitution_turnover_rate_significant'] = all_table['res'].apply(lambda res: res.pvalues['intitution_turnover_rate']  <= 0.1 )

In [28]:
print('資券顯著：', all_table['is_margin_short_ratio_significant'].sum())
print('法人顯著：', all_table['is_intitution_turnover_rate_significant'].sum())
print('皆顯著：', (
    all_table['is_margin_short_ratio_significant'] & all_table['is_intitution_turnover_rate_significant']
).sum())

資券顯著： 125
法人顯著： 291
皆顯著： 37


In [29]:
all_significant_table = all_table[all_table['is_margin_short_ratio_significant'] & all_table['is_intitution_turnover_rate_significant']]

In [42]:
both_significant = all_table['is_margin_short_ratio_significant'] & all_table['is_intitution_turnover_rate_significant']

In [46]:
output = all_table[all_table['is_margin_short_ratio_significant'] ^ both_significant]

In [53]:
output['obs_num']= output['res'].apply(lambda res: res.nobs)
output.to_pickle("all_significant_table.pkl")

In [54]:
output

,code,res,name,type,aviable_t,mean_size,new_industry,industry,is_margin_short_ratio_significant,is_intitution_turnover_rate_significant,obs_num
206,3591,<statsmodels.regression.linear_model.Regressio...,艾笛森,all,5,2019.990588,光電業,光電/ IO,True,False,1275.0
272,2880,<statsmodels.regression.linear_model.Regressio...,華南金,all,5,224759.662745,金融業,金融控股,True,False,1275.0
1730,2528,<statsmodels.regression.linear_model.Regressio...,皇普,all,5,4686.052549,建材營造,建設,True,False,1275.0
1478,2458,<statsmodels.regression.linear_model.Regressio...,義隆,all,5,31702.098503,半導體,半導體,True,False,1269.0
329,2637,<statsmodels.regression.linear_model.Regressio...,慧洋-KY,all,5,25518.819608,航運業,海運,True,False,1275.0
...,...,...,...,...,...,...,...,...,...,...,...
3083,9914,<statsmodels.regression.linear_model.Regressio...,美利達,all,1,58572.770980,其他,其他,True,False,1275.0
2750,3088,<statsmodels.regression.linear_model.Regressio...,艾訊,all,1,4500.046275,電腦及週邊,主機板,True,False,1275.0
1211,1308,<statsmodels.regression.linear_model.Regressio...,亞聚,all,1,11595.503529,塑膠工業,石化,True,False,1275.0
2567,8928,<statsmodels.regression.linear_model.Regressio...,鉅明,all,1,1467.521569,其他,其他,True,False,1275.0


In [33]:
print('all var : ', len(all_x_variable_columns))
all_significant_pivot_table = all_significant_table.pivot_table(
    index=['new_industry'], 
    aggfunc=np.mean,
    values=['aviable_t' , 'mean_size'],
)\
.sort_values(
    'aviable_t', ascending=False
)
all_significant_pivot_table

all var :  8


,aviable_t,mean_size
new_industry,,
食品工業,6.000000,1700.524706
油電燃氣業,6.000000,6321.593725
生技醫療,5.000000,5849.094902
電機機械,4.666667,3993.756601
貿易百貨,4.500000,1017.696078
電子商務,4.500000,5964.840784
半導體,4.250000,7370.423922
文化創意業,4.000000,519.959216
其他電子業,4.000000,1383.284706


In [34]:
all_significant_table.pivot_table(
    index=['new_industry', 'industry'], 
    values=['aviable_t' , 'mean_size'],
    aggfunc=np.mean
)

aviable_t     mean_size
new_industry industry                         
光電業          光電/ IO         3.00   2964.706667
             電子設備           3.00   2042.487843
其他           汽車製造           3.00   1016.500392
其他電子業        電子設備           4.00   1383.284706
化學工業         染料及顏料          4.00   2848.086275
             樹脂             3.00   1612.099608
半導體          半導體            4.25   7370.423922
建材營造         建設             3.00   6376.651373
             營造             3.00   3359.149489
文化創意業        玻璃陶瓷           4.00    519.959216
橡膠工業         輪胎             3.00   1037.276863
油電燃氣業        公用事業           6.00   6321.593725
生技醫療         醫療耗材           5.00   5849.094902
紡織纖維         人纖             4.00   5788.847059
             成衣             2.00  10973.817255
             染整             4.00   1930.987392
貿易百貨         百貨批發           6.00    975.538039
             石化             3.00   1059.854118
通信網路業        網路設備           4.00   2356.008627
             資訊通路           3.00   9946.545098
金融業          產險業            3.00   7301.620392
鋼鐵工業         金屬基本           3.00   1044.429020
電子商務         軟體服務           5.00   1588.912941
             通訊設備           4.00  10340.768627
電子零組件        主機板            4.00  15872.538824
             資訊通路           3.00   1351.374118
             電子零組件          5.00   3968.018039
電機機械         汽車組件           5.00   5603.161176
             營造             4.00    774.947451
食品工業         其他食品           6.00   1700.524706

In [35]:
all_table.pivot_table(
    index=['new_industry', 'industry'], 
    values=['aviable_t' , 'mean_size'],
    aggfunc=np.mean
)

aviable_t      mean_size
new_industry industry                          
光電業          主機板        2.000000   21343.393725
             光電/ IO     2.357143   12070.753810
             家電製品       2.000000     701.030733
             消費性電子      2.400000   62956.354980
             電子設備       1.857143    2869.000434
...                          ...            ...
電腦及週邊        電子零組件      2.095238    7419.327609
食品工業         其他食品       2.555556   10757.353638
             油脂飼料       2.200000   16249.486588
             罐頭加工       1.000000    4021.747451
             飲料乳品       2.000000  106515.680000

[183 rows x 2 columns]

In [36]:
all_significant_table

,code,res,name,type,aviable_t,mean_size,new_industry,industry,is_margin_short_ratio_significant,is_intitution_turnover_rate_significant,obs_num
884,5288,<statsmodels.regression.linear_model.Regressio...,豐祥-KY,all,8,7402.869804,電機機械,汽車組件,True,True,1275.0
2687,6485,<statsmodels.regression.linear_model.Regressio...,點序,all,8,2244.040000,半導體,半導體,True,True,1275.0
3287,4736,<statsmodels.regression.linear_model.Regressio...,泰博,all,7,13472.272157,生技醫療,醫療耗材,True,True,1275.0
3086,9918,<statsmodels.regression.linear_model.Regressio...,欣天然,all,6,6321.593725,油電燃氣業,公用事業,True,True,1275.0
1268,1236,<statsmodels.regression.linear_model.Regressio...,宏亞,all,6,1700.524706,食品工業,其他食品,True,True,1275.0
2732,2916,<statsmodels.regression.linear_model.Regressio...,滿心,all,6,975.538039,貿易百貨,百貨批發,True,True,1275.0
1490,2462,<statsmodels.regression.linear_model.Regressio...,良得電,all,5,3968.018039,電子零組件,電子零組件,True,True,1275.0
3188,4138,<statsmodels.regression.linear_model.Regressio...,曜亞,all,5,1535.186667,生技醫療,醫療耗材,True,True,1275.0
3443,3687,<statsmodels.regression.linear_model.Regressio...,歐買尬,all,5,1588.912941,電子商務,軟體服務,True,True,1275.0
3185,4401,<statsmodels.regression.linear_model.Regressio...,東隆興,all,4,5788.847059,紡織纖維,人纖,True,True,1275.0


# 有興趣的投資人指標

In [ ]:
expect_col = ['margin_short_ratio', 'intitution_turnover_rate']
aviable_t_list = []
for res in res_list:
    aviable_t = (res['res'].pvalues[expect_col] < 0.1).sum()
    if aviable_t > 0:
        code = res['code']
        stock_data = all_data.query('code == @code')[["code","name","new_industry","industry"]].dropna()
        stock_info = stock_data.iloc[-1, :4]
        #print(" | ".join(stock_data.iloc[-1, :4].values), aviable_t)
        aviable_t_list.append({
            'code': code,
            'name': stock_info['name'],
            'new_industry': stock_info['new_industry'],
            'industry': stock_info['industry'],
            'aviable_t': aviable_t
        })
aviable_t_table = pd.DataFrame(aviable_t_list)

In [ ]:
aviable_t_table.pivot_table(index=['new_industry'], aggfunc=sum).sort_values(
    'aviable_t', ascending=False
)

In [ ]:
target_industry = aviable_t_table[aviable_t_table['new_industry'].isin(
    ['電子零組件', '半導體', '電機機械', '建材營造'])
]
target_industry.pivot_table(index=['new_industry', 'industry'], aggfunc=sum)

In [ ]:
price_data

# 單隻

In [ ]:
code = "2498"

stock_data = all_data.query('code == @code')
stock_data = stock_data.ffill().dropna()
stock_data['return_t1'] = stock_data['return'].shift(-1)
stock_data = stock_data.dropna()

try:
    
    stock_data['PB_TSE'] = stock_data['PB_TSE'].astype(float)
    stock_data['YOY'] = stock_data['YOY'].astype(float)
    stock_data['TAR'] = stock_data['TAR'].astype(float)
        
        
    ## 資券比
    stock_data['short_margin_ratio'] = stock_data['short_margin_ratio'].astype(float)
    def convert(x):
        if x == 0 :
            return 0
        else:
            return 1/(x)*10000
    stock_data['margin_short_ratio'] = stock_data['short_margin_ratio'].apply(convert)

    ## 合計週轉率
    stock_data['intitution_turnover_rate']  = stock_data[['trust_turnover_rate', 'broker_turnover_rate', 'short_margin_ratio']].sum(1)

except:
    print("有問題：", code)
    

y = stock_data['return_t1'].values.reshape((-1, 1))
x = stock_data[[
    'return', 
    'size', 
    'PB_TSE',
#     'ROA',
    'YOY', 
    'TAR',
    'turn_over', 
    'margin_short_ratio', 
    'intitution_turnover_rate',
]]

model = sm.OLS(y,  sm.add_constant(x,  prepend=False))
res = model.fit()       # Fit model
print(res.summary())   # Summarize model

aviable_t = (res.pvalues[:-1] < 0.1).sum()
if aviable_t > 2:
    print(stock_data.iloc[0, :3])
    print(aviable_t)
    print('----------')

In [ ]:
code = "2201"
stock_data = all_data.query('code == @code')
stock_data = stock_data.ffill().dropna()
stock_data['return_t1'] = stock_data['return'].shift(-1)
stock_data = stock_data.dropna()

try:
    
    stock_data['PB_TSE'] = stock_data['PB_TSE'].astype(float)
    stock_data['YOY'] = stock_data['YOY'].astype(float)
    stock_data['TAR'] = stock_data['TAR'].astype(float)
        
        
    ## 資券比
    stock_data['short_margin_ratio'] = stock_data['short_margin_ratio'].astype(float)
    def convert(x):
        if x == 0 :
            return 0
        else:
            return 1/(x)*10000
    stock_data['margin_short_ratio'] = stock_data['short_margin_ratio'].apply(convert)

    ## 合計週轉率
    stock_data['intitution_turnover_rate']  = stock_data[['trust_turnover_rate', 'broker_turnover_rate', 'short_margin_ratio']].sum(1)

except:
    print("有問題：", code)
    

y = stock_data['return_t1'].values.reshape((-1, 1))
x = stock_data[[
    'return', 
    'size', 
    'PB_TSE',
#     'ROA',
    'YOY', 
    'TAR',
    'turn_over', 
    'margin_short_ratio', 
    'intitution_turnover_rate',
]]

model = sm.OLS(y,  sm.add_constant(x,  prepend=False))
res = model.fit()       # Fit model
print(res.summary())   # Summarize model

aviable_t = (res.pvalues[:-1] < 0.1).sum()
if aviable_t > 2:
    print(stock_data.iloc[0, :3])
    print(aviable_t)
    print('----------')

In [ ]:
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.598
Date:                Mon, 28 Feb 2022   Prob (F-statistic):              0.121
Time:                        17:15:48   Log-Likelihood:                -2597.6
No. Observations:                1256   AIC:                             5213.
Df Residuals:                    1247   BIC:                             5259.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
return                       0.0018      0.029      0.064      0.949      -0.055       0.059
size                     -1.868e-05   1.58e-05     -1.185      0.236   -4.96e-05    1.23e-05
PB_TSE                       0.4055      0.624      0.650      0.516      -0.818       1.630
YOY                         -0.0113      0.008     -1.345      0.179      -0.028       0.005
TAR                         -0.0008      0.009     -0.096      0.923      -0.018       0.016
turn_over                    0.3170      0.105      3.015      0.003       0.111       0.523
margin_short_ratio       -2.458e-07   1.19e-06     -0.206      0.836   -2.58e-06    2.09e-06
intitution_turnover_rate    -0.0107      0.025     -0.421      0.674      -0.060       0.039
const                        0.2918      0.583      0.501      0.617      -0.852       1.436
==============================================================================
Omnibus:                      252.497   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2497.824
Skew:                           0.627   Prob(JB):                         0.00
Kurtosis:                       9.794   Cond. No.                     6.91e+05
==============================================================================


# 讀取資料

In [ ]:
price_data = pd.read_csv('./taiwan-index-price-data.txt', encoding='cp950')

In [ ]:
price_data

# 資料基本欄位檢查

In [ ]:
price_data.info()

In [ ]:
print(price_data['證券代碼'].drop_duplicates())
print('-----------------------------')
print(price_data['簡稱'].drop_duplicates())

# 查詢資料

In [ ]:
price_data['年月日'] == 20000104

##### 為什麼找不到想要的代碼

In [ ]:
target_data = price_data[price_data['證券代碼'] == 'M1800']
target_data

In [ ]:
price_data['證券代碼'].drop_duplicates().values

In [ ]:
def clearSpace(value):
    return value.strip()

In [ ]:
price_data['證券代碼'] = price_data['證券代碼'].apply(clearSpace)
price_data['簡稱'] = price_data['簡稱'].apply(clearSpace)

#### 再來找看看

In [ ]:
target_data = price_data[price_data['證券代碼'] == 'M1800']
target_data

## index 跑掉了？

In [ ]:
target_data = target_data.reset_index(drop=True)
target_data

# 新增報酬率欄位

In [ ]:
target_data['收盤價(元)']

In [ ]:
target_data['ret'] = target_data['收盤價(元)'].diff(1)/target_data['收盤價(元)'].shift(1)
target_data

In [ ]:
target_data = target_data.dropna()

# 基本敘述性統計

In [ ]:
target_data.describe()

## 偏態 與 峰態

In [ ]:
from scipy import stats

In [ ]:
## 偏態
skew = stats.skew(target_data['收盤價(元)'])
## 峰態 fisher=False 是 Eviews 作法
kurtosis = stats.kurtosis(target_data['收盤價(元)'], fisher=False)

print('skew      :  {}'.format(skew))
print('kurtosis  :  {}'.format(kurtosis))

# 相關係數表

In [ ]:
target_data.corr()